RGBA - cât roșu, verde, albastru am într-o imagine (de la 0 la 255).
A - vine de la alpha. Aka: transparența culorii.
În Pillow, roșu este reprezentat ca (255, 0, 0, 255). Adică nu am deloc componentele de verde și albastru, iar culoarea este total opacă. 
Alb: (255, 255, 255).
Negru: (0, 0, 0, 255).

Mai jos avem câteva funcții utile din API-ul de PILLOW.

In [1]:
from PIL import ImageColor

# We can use getcolor to get the actual rgba tuple associated with a color given by name
# The name passed as parameter is case insensitive
rgba_cfb = ImageColor.getcolor('CornFlowerBlue', 'RGBA')
rgba_choc = ImageColor.getcolor('chocolate', 'RGBA')

print("CornFlowerBlue: ", rgba_cfb)
print("chocolate: ", rgba_choc)

CornFlowerBlue:  (100, 149, 237, 255)
chocolate:  (210, 105, 30, 255)


Originea imaginii: (0, 0) - colțul din stânga sus al ecranului.
Coordonate:
    - x: crește de la stânga la dreapta
    - y crește în jos
    
În continuare mai avem câteva exemple de cum să folosim manipularea de imagini cu pillow.

In [8]:
# The name of the module is PIL and not pillow in order to ensure backward compatibili5ty with another module
# which is older and called Python Imaging Library
from PIL import Image
import os

# this changes the working directory
# to be used if the image is at a different path in the file system
os.chdir('C:\\Users\mgaman\\Downloads')

# This saves in trumpImg the reference to our image file.
trumpImg = Image.open('trumpish.jpg')

# Print the size of the image
print("The size of trumpImg is: ", trumpImg.size)
# the name of the image file
print("The name of the image file is: ", trumpImg.filename)
# format
print("Format: ", trumpImg.format)
# format description
print("Format description: ", trumpImg.format_description)
# save another image based on this object
trumpImg.save("trumpish_copy.png")

The size of trumpImg is:  (1296, 730)
The name of the image file is:  trumpish.jpg
Format:  JPEG
Format description:  JPEG (ISO 10918)


Folosind Pillow, putem chiar să ne creem imagini noi de la zero.
Un mic tutorial în acest sens avem mai jos:

In [11]:
from PIL import Image

# (150, 300) here specifies the size of the image given by width and height
# if no color is provided as the third parametr, we wioll obtain a transparent image
presidential_orange = Image.new('RGBA', (150, 300), 'orange')
presidential_orange.save('orange_trump.jpg')

Să învățăm acum cum să:
    - tăiem imagini (crop)
    - copiem imagini (sau bucăți din acestea) peste alte imagini
        - de notat că, în ciuda numelor, metodele de copy și paste din pillow nu au nimic de a face cu
        clipboard-ul.
        - mare atenție și la imaginile cu pixeli transparenți (cu alpha = 0). Acolo pixelii sunt tratați ca 
        opaci (alb) la paste.
    - resize
    - rotate & flip

In [30]:
from PIL import Image
import os

os.chdir('C:\\Users\mgaman\\Downloads')

######################### CROP #########################
trumpImg = Image.open('trumpish.jpg')
# The image cropped will be associated with another object...
croppedTrump = trumpImg.crop((400, 150, 1090, 530))
# ... and we have to save the contents of this object in an image file.
# Thus the old image will stay the same (will not be changed by this operation)
croppedTrump.save('trumpish_cropped.jpg')

######################## Copy & paste ######################
# image to be cropped
trumpImg2 = Image.open('donald-trump-simpsons.jpg')
(w, h) = trumpImg2.size

# We want to crop the head
crop_xi, crop_xf = 150, int(w/2) + 50
crop_w = crop_xf - crop_xi
c_simps_trump = trumpImg2.crop((crop_xi, 50, crop_xf, int(h/2)))
c_simps_trump.save('trumpish_cropped_simpsons.jpg')

new_w, new_h = 600, 400
presidential_orange = Image.new('RGBA', (new_w, new_h), 'orange')

# Now add 3 Trump images in the new orange image
no_copies = 3
# How much space is left after all the images are copied one near the other
space = int((new_w - no_copies * crop_w) / 3)
# how much of this space receives each image
# (this is the space left between the copies pasted on the orange background)
offset_x = int(space / no_copies)
for i in range(no_copies):
    presidential_orange.paste(c_simps_trump, (i * (crop_w + offset_x), 0))
#... and save
presidential_orange.save('orange_trump.jpg')

########################### RESIZE #########################
w_trumpImg, h_trumpImg = trumpImg.size
smallTrump = trumpImg.resize((int(w_trumpImg/2), int(h_trumpImg/2)))
smallTrump.save('small_trump.jpg')

########################### ROTATE AND FLIP #########################
# Paste all the rotations in a single image
rick_morty_sample = Image.open('rick_morty.jpg')

w_rm_sample, h_rm_sample = rick_morty_sample.size
w_multi_rot = w_rm_sample + h_rm_sample
h_multi_rot = w_multi_rot

multi_rot = Image.new('RGBA', (w_multi_rot, h_multi_rot), 'skyblue')
# Let's observe what happens with the size of the image rotated.
# It keeps the same proportions as its source. 
# The padding is done using black opaque pixels 
# (on windows; on os-x there are transparent pixels used instead).
# In order to fix this, call the method with the expand parameter set to True.
multi_rot.paste(rick_morty_sample.rotate(0, expand=True), (0, 0))
multi_rot.paste(rick_morty_sample.rotate(90, expand=True), (w_rm_sample, 0))
multi_rot.paste(rick_morty_sample.rotate(180, expand=True), ( h_rm_sample,h_multi_rot - h_rm_sample))
multi_rot.paste(rick_morty_sample.rotate(270, expand=True), (0, h_rm_sample))

multi_rot.save('multi_rot.jpg')

# Now let's FLIP
south_park_sample = Image.open('south_park.jpg')
w_sp, h_sp = south_park_sample.size
south_park_small = south_park_sample.resize((int(w_sp/2), int(h_sp/2)))

multi_flips = Image.new('RGBA', (w_sp, h_sp), 'olive')
multi_flips.paste(south_park_small, (0, 0))
multi_flips.paste(south_park_small.transpose(Image.FLIP_LEFT_RIGHT), (int(w_sp/2), 0))
multi_flips.paste(south_park_small, (0, int(h_sp/2)))
multi_flips.paste(south_park_small.transpose(Image.FLIP_TOP_BOTTOM), (int(w_sp/2), (int(h_sp/2))))
multi_flips.save('multi_flips.jpg')